In [138]:
import torch
import numpy as np
import yfinance as yf
import statsmodels as sm
import pandas as pd
from pytorch_forecasting import TimeSeriesDataSet

In [139]:
tickers = ['^GSPC', '^DJI', 'NQ=F', 'EURUSD=X', 'GC=F']
data = yf.download(tickers, start="2015-01-01", end="2025-01-01", group_by='ticker')

[*********************100%***********************]  5 of 5 completed


In [140]:
closing_price = pd.DataFrame()

In [141]:
for ticker in tickers:
    closing_price[ticker] = data[ticker]['Close']

In [142]:
closing_price

,^GSPC,^DJI,NQ=F,EURUSD=X,GC=F
Date,,,,,
2015-01-01,NaN,NaN,NaN,1.209863,NaN
2015-01-02,2058.199951,17832.990234,4214.25,1.208941,1186.000000
2015-01-05,2020.579956,17501.650391,4161.75,1.194643,1203.900024
2015-01-06,2002.609985,17371.640625,4102.25,1.193902,1219.300049
2015-01-07,2025.900024,17584.519531,4151.50,1.187536,1210.599976
...,...,...,...,...,...
2024-12-25,NaN,NaN,NaN,1.040258,NaN
2024-12-26,6037.589844,43325.800781,22008.00,1.039955,2638.800049
2024-12-27,5970.839844,42992.210938,21698.50,1.042318,2617.199951


In [143]:
closing_price.dropna(inplace=True)

In [144]:
log_returns = np.log(closing_price/closing_price.shift(-1))

In [145]:
log_returns.dropna(inplace=True)

In [146]:
log_returns

,^GSPC,^DJI,NQ=F,EURUSD=X,GC=F
Date,,,,,
2015-01-02,0.018447,0.018755,0.012536,0.011897,-0.014980
2015-01-05,0.008933,0.007456,0.014400,0.000621,-0.012711
2015-01-06,-0.011563,-0.012180,-0.011934,0.005346,0.007161
2015-01-07,-0.017730,-0.018221,-0.019264,0.003320,0.001819
2015-01-08,0.008439,0.009567,0.007471,0.003379,-0.006270
...,...,...,...,...,...
2024-12-23,-0.010982,-0.009050,-0.012574,0.002615,-0.002943
2024-12-24,0.000406,-0.000664,0.000931,0.000603,-0.007150
2024-12-26,0.011117,0.007729,0.014163,-0.002270,0.008219


In [147]:
def getVariance(windowsize):
    volatility = np.sqrt((log_returns**2).rolling(windowsize).sum())
    return volatility

In [148]:
volatility = getVariance(5)

In [149]:
volatility

,^GSPC,^DJI,NQ=F,EURUSD=X,GC=F
Date,,,,,
2015-01-02,NaN,NaN,NaN,NaN,NaN
2015-01-05,NaN,NaN,NaN,NaN,NaN
2015-01-06,NaN,NaN,NaN,NaN,NaN
2015-01-07,NaN,NaN,NaN,NaN,NaN
2015-01-08,0.030649,0.031293,0.030559,0.013891,0.021906
...,...,...,...,...,...
2024-12-23,0.034455,0.030115,0.054988,0.015789,0.023226
2024-12-24,0.017062,0.014887,0.040845,0.015657,0.024117
2024-12-26,0.020346,0.016770,0.042940,0.007569,0.019027


In [150]:
data = []
for i, ticker in enumerate(tickers):
    temp_df = pd.DataFrame({
        "time_idx": np.arange(len(log_returns)),  # Временные индексы
        "group": i,  # Группа (тикер)
        "log_returns": log_returns[ticker].values,  # Логарифмические доходности
        "volatility": volatility[ticker].values,  # Волатильность (целевая переменная)
    })
    data.append(temp_df)

# Объединяем все данные в один DataFrame
data = pd.concat(data)


In [151]:
data.fillna(0,inplace=True)

In [152]:
data.isnull().sum()

time_idx       0
group          0
log_returns    0
volatility     0
dtype: int64

In [153]:
data = data.reset_index(drop=True)

In [154]:
data

,time_idx,group,log_returns,volatility
0,0,0,0.018447,0.000000
1,1,0,0.008933,0.000000
2,2,0,-0.011563,0.000000
3,3,0,-0.017730,0.000000
4,4,0,0.008439,0.030649
...,...,...,...,...
12540,2504,4,-0.002943,0.023226
12541,2505,4,-0.007150,0.024117
12542,2506,4,0.008219,0.019027
12543,2507,4,0.004250,0.013586


In [155]:
dataset = TimeSeriesDataSet(
    data=data,
    group_ids=["group"],  
    target="volatility",   
    time_idx="time_idx",  
    max_encoder_length=5,  
    max_prediction_length=1,  
    time_varying_known_reals=["log_returns"], 
    time_varying_unknown_reals=["volatility"],
    target_normalizer=None,
)

TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.